# 0 : Import the required libraries and define the Salesforce connection 

In [10]:
from simple_salesforce import Salesforce
import pandas as pd

#connect to salesforce
config = pd.read_json('config.json')
source_org = Salesforce(username=config['source']['username'], password=config['source']['password'], security_token=config['source']['token'])
target_org = Salesforce(username=config['target']['username'], password=config['target']['password'], security_token=config['target']['token'])
connected = source_org.session_id is not None
print('Connected to Source: '+str(connected))
connected = target_org.session_id is not None
print('Connected to Target: '+str(connected))

Connected to Source: True
Connected to Target: True


# 1 : Query the data from Source Org

In [11]:
#query data
contacts_dict = source_org.query_all("SELECT Id, LastName, Email, Phone, AccountId FROM Contact")
account_dict = source_org.query_all("SELECT Id, Name, Phone FROM Account")
#convert to dataframes
accounts_df = pd.DataFrame(account_dict['records']).drop(columns='attributes')
contacts_df = pd.DataFrame(contacts_dict['records']).drop(columns='attributes')

print('Contacts:')
print(contacts_df)
print('Accounts:')
print(accounts_df)


Contacts:
                    Id   LastName                      Email            Phone  \
0   0038b000039sgwJAAQ   Gonzalez              rose@edge.com   (512) 757-6000   
1   0038b000039sgwKAAQ     Forbes              sean@edge.com   (512) 757-6000   
2   0038b000039sgwLAAQ     Rogers     jrogers@burlington.com   (336) 222-7000   
3   0038b000039sgwMAAQ  Stumuller            pat@pyramid.net   (014) 427-4427   
4   0038b000039sgwNAAQ      Young      a_young@dickenson.com   (785) 241-6200   
5   0038b000039sgwOAAQ       Barr   barr_tim@grandhotels.com   (312) 596-1000   
6   0038b000039sgwPAAQ       Bond  bond_john@grandhotels.com   (312) 596-1000   
7   0038b000039sgwQAAQ    Pavlova           spavlova@uog.com   (212) 842-5500   
8   0038b000039sgwRAAQ      Boyle             lboyle@uog.com   (212) 842-5500   
9   0038b000039sgwSAAQ       Levy      b.levy@expressl&t.net   (503) 421-7800   
10  0038b000039sgwTAAQ      Davis     j.davis@expressl&t.net   (503) 421-7800   
11  0038b000039sgw

# 2 : Clean and Insert the Parent Object data into Salesforce Target Org

In [12]:
accounts_df.rename(columns={'Id':'Legacy_Id__c'}, inplace=True)
#convert to dictionary
accounts_records = accounts_df.to_dict('records')
# bulk insert into target org
result = target_org.bulk.Account.insert(accounts_records)
print(result)

[{'success': False, 'created': False, 'id': None, 'errors': [{'statusCode': 'DUPLICATE_VALUE', 'message': 'duplicate value found: Legacy_Id__c duplicates value on record with id: 0018b00002XIoMdAAL', 'fields': []}]}, {'success': False, 'created': False, 'id': None, 'errors': [{'statusCode': 'DUPLICATE_VALUE', 'message': 'duplicate value found: Legacy_Id__c duplicates value on record with id: 0018b00002XIoMeAAL', 'fields': []}]}, {'success': False, 'created': False, 'id': None, 'errors': [{'statusCode': 'DUPLICATE_VALUE', 'message': 'duplicate value found: Legacy_Id__c duplicates value on record with id: 0018b00002XIoMfAAL', 'fields': []}]}, {'success': False, 'created': False, 'id': None, 'errors': [{'statusCode': 'DUPLICATE_VALUE', 'message': 'duplicate value found: Legacy_Id__c duplicates value on record with id: 0018b00002XIoMgAAL', 'fields': []}]}, {'success': False, 'created': False, 'id': None, 'errors': [{'statusCode': 'DUPLICATE_VALUE', 'message': 'duplicate value found: Legacy

# 3 : Create a intersection dataframe to match Source and Target Id's 

In [13]:
#Id and Legacy_Id__c are going to be ObjectId
target_account_dict = target_org.query_all("SELECT Id, Legacy_Id__c FROM Account")
target_accounts = pd.DataFrame(target_account_dict['records']).drop(columns='attributes')
print(target_accounts)

                    Id        Legacy_Id__c
0   0018b00002XIoNYAA1                None
1   0018b00002XIoNZAA1                None
2   0018b00002XIoNXAA1                None
3   0018b00002XIoNDAA1                None
4   0018b00002XIoNEAA1                None
5   0018b00002XIoNFAA1                None
6   0018b00002XIoNSAA1                None
7   0018b00002XIoN8AAL                None
8   0018b00002XIoN9AAL                None
9   0018b00002XIoNAAA1                None
10  0018b00002XIoG9AAL                None
11  0018b00002WaM71AAF                None
12  0018b00002WaM9vAAF                None
13  0018b00002WaMGDAA3                None
14  0018b00002WaMAPAA3                None
15  0018b00002WaMAZAA3                None
16  0018b00002WaMGIAA3                None
17  0018b00002WlmLdAAJ                None
18  0018b00002WlmLeAAJ                None
19  0018b00002WlmLfAAJ                None
20  0018b00002WlmLgAAJ                None
21  0018b00002WlmLhAAJ                None
22  0018b00

# 4 : Contact Union Interception : Join both dataframes with a LEFT JOIN to add the equivalent Id from Interception

In [14]:
contacts_df = contacts_df.merge(target_accounts, how='left', left_on='AccountId', right_on='Legacy_Id__c')
print(contacts_df)

                  Id_x   LastName                      Email            Phone  \
0   0038b000039sgwJAAQ   Gonzalez              rose@edge.com   (512) 757-6000   
1   0038b000039sgwKAAQ     Forbes              sean@edge.com   (512) 757-6000   
2   0038b000039sgwLAAQ     Rogers     jrogers@burlington.com   (336) 222-7000   
3   0038b000039sgwMAAQ  Stumuller            pat@pyramid.net   (014) 427-4427   
4   0038b000039sgwNAAQ      Young      a_young@dickenson.com   (785) 241-6200   
5   0038b000039sgwOAAQ       Barr   barr_tim@grandhotels.com   (312) 596-1000   
6   0038b000039sgwPAAQ       Bond  bond_john@grandhotels.com   (312) 596-1000   
7   0038b000039sgwQAAQ    Pavlova           spavlova@uog.com   (212) 842-5500   
8   0038b000039sgwRAAQ      Boyle             lboyle@uog.com   (212) 842-5500   
9   0038b000039sgwSAAQ       Levy      b.levy@expressl&t.net   (503) 421-7800   
10  0038b000039sgwTAAQ      Davis     j.davis@expressl&t.net   (503) 421-7800   
11  0038b000039sgwUAAQ      

# 5 : Clean the data by dropping no needed columns and rename the ContactId to Legacy__Id__c

In [15]:
contacts_df.drop(columns=['AccountId','Legacy_Id__c'], inplace=True)
contacts_df.rename(columns={'Id_x':'Legacy_Id__c'}, inplace=True)
contacts_df.rename(columns={'Id_y':'AccountId'}, inplace=True)
print(contacts_df)

          Legacy_Id__c   LastName                      Email            Phone  \
0   0038b000039sgwJAAQ   Gonzalez              rose@edge.com   (512) 757-6000   
1   0038b000039sgwKAAQ     Forbes              sean@edge.com   (512) 757-6000   
2   0038b000039sgwLAAQ     Rogers     jrogers@burlington.com   (336) 222-7000   
3   0038b000039sgwMAAQ  Stumuller            pat@pyramid.net   (014) 427-4427   
4   0038b000039sgwNAAQ      Young      a_young@dickenson.com   (785) 241-6200   
5   0038b000039sgwOAAQ       Barr   barr_tim@grandhotels.com   (312) 596-1000   
6   0038b000039sgwPAAQ       Bond  bond_john@grandhotels.com   (312) 596-1000   
7   0038b000039sgwQAAQ    Pavlova           spavlova@uog.com   (212) 842-5500   
8   0038b000039sgwRAAQ      Boyle             lboyle@uog.com   (212) 842-5500   
9   0038b000039sgwSAAQ       Levy      b.levy@expressl&t.net   (503) 421-7800   
10  0038b000039sgwTAAQ      Davis     j.davis@expressl&t.net   (503) 421-7800   
11  0038b000039sgwUAAQ      

# 6 : Insert the Child object into Salesforce

In [16]:
#Convert dataframe to dictionary
contacts_to_insert = contacts_df.to_dict('records')
#bulk insert into target org
result = target_org.bulk.Contact.insert(contacts_to_insert)
print(result)

[{'success': True, 'created': True, 'id': '0038b00003CuWfAAAV', 'errors': []}, {'success': True, 'created': True, 'id': '0038b00003CuWfBAAV', 'errors': []}, {'success': True, 'created': True, 'id': '0038b00003CuWfCAAV', 'errors': []}, {'success': True, 'created': True, 'id': '0038b00003CuWfDAAV', 'errors': []}, {'success': True, 'created': True, 'id': '0038b00003CuWfEAAV', 'errors': []}, {'success': True, 'created': True, 'id': '0038b00003CuWfFAAV', 'errors': []}, {'success': True, 'created': True, 'id': '0038b00003CuWfGAAV', 'errors': []}, {'success': True, 'created': True, 'id': '0038b00003CuWfHAAV', 'errors': []}, {'success': True, 'created': True, 'id': '0038b00003CuWfIAAV', 'errors': []}, {'success': False, 'created': False, 'id': None, 'errors': [{'statusCode': 'INVALID_EMAIL_ADDRESS', 'message': 'Email: invalid email address: b.levy@expressl&amp;t.net', 'fields': ['Email']}]}, {'success': False, 'created': False, 'id': None, 'errors': [{'statusCode': 'INVALID_EMAIL_ADDRESS', 'me